In [36]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, log_loss, multilabel_confusion_matrix, f1_score, average_precision_score

Dataset is split 80%/20% as training/testing data Multi - Classification model ROC/AUC is used to evaluate predictionsModels used -DecisionTreeClassifier -RandomForestClassifier -XGBoostClassifier

In [37]:
%store -r df #moving dataset from previous notebook

no stored variable #moving dataset from previous notebook


In [38]:
from sklearn.model_selection import train_test_split
X = df[['status', 'location', 'amount_mean', 'operation_mean', 'incoming_mean', 'outgoing_mean', 'incoming_max',
'outgoing_max', 'incoming_min', 'outgoing_min', 'age', 'Generation', 'Life-Cycle', 'Female', 'Male', 
'email_types', 'type_mean_email', 'credit-amount-gr', 'loan_payment_mean', 'Student', 'Employed', 'Resident']]
Y = df.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [39]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 682 entries, 0 to 681
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   status             682 non-null    int64  
 1   location           682 non-null    int64  
 2   amount_mean        682 non-null    float64
 3   operation_mean     682 non-null    float64
 4   incoming_mean      682 non-null    float64
 5   outgoing_mean      682 non-null    float64
 6   incoming_max       682 non-null    float64
 7   outgoing_max       682 non-null    float64
 8   incoming_min       682 non-null    float64
 9   outgoing_min       682 non-null    float64
 10  age                682 non-null    float64
 11  Generation         682 non-null    int64  
 12  Life-Cycle         682 non-null    int64  
 13  Female             682 non-null    int64  
 14  Male               682 non-null    int64  
 15  email_types        682 non-null    int64  
 16  type_mean_email    682 non

# Random Forest

In [109]:
rf_tree = RandomForestClassifier()
rf_tree.fit(X_train, y_train)
y_pred = rf_tree.predict_proba(X_test)

print('Logarithmic Loss:', metrics.log_loss(y_test, y_pred))

Logarithmic Loss: 0.21044798065017797


/Users/amihailovs/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Optimization

### Grid search

In [121]:
rf_params = {'min_samples_split': [2, 3, 4, 5, 6], 
              'min_samples_leaf': [4, 6, 8, 12, 15],
              'n_estimators': [100, 150, 170, 200, 300],
              'max_depth': [10, 20, 30, 50, 70]}

kf = KFold(n_splits=5)

In [122]:
grid_searcher_rf = GridSearchCV(estimator=rf_tree, param_grid=rf_params, 
                             cv=kf, verbose=1, n_jobs=4)

In [123]:
grid_searcher_rf.fit(X_train, y_train)

Fitting 5 folds for each of 625 candidates, totalling 3125 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   25.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   58.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done 3125 out of 3125 | elapsed:  6.4min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                      

In [124]:
grid_searcher_rf.best_params_, grid_searcher_rf.best_score_

({'max_depth': 70,
  'min_samples_leaf': 4,
  'min_samples_split': 6,
  'n_estimators': 150},
 0.9357798165137615)

In [125]:
rf_tree_opt = RandomForestClassifier(
    n_estimators=150,
    min_samples_split=6,
    min_samples_leaf=4,
    max_features='log2',
    max_depth=70,
    bootstrap=True
)

In [126]:
rf_tree_opt.fit(X_train, y_train)
rf_tree_opt.predict_proba(X_test)

array([[2.46774892e-02, 7.72893218e-02, 8.60558682e-01, 3.74745070e-02],
       [4.61111111e-03, 1.44922670e-02, 9.71521775e-01, 9.37484737e-03],
       [1.58838134e-02, 3.84624635e-02, 9.28278469e-01, 1.73752544e-02],
       [2.41558442e-03, 4.95238095e-03, 9.89298701e-01, 3.33333333e-03],
       [1.66137566e-02, 9.59292559e-02, 8.69513357e-01, 1.79436304e-02],
       [2.85714286e-03, 1.15396825e-02, 9.76555556e-01, 9.04761905e-03],
       [2.05484571e-01, 3.12684519e-01, 1.52503608e-01, 3.29327302e-01],
       [1.33333333e-03, 2.66666667e-03, 9.96000000e-01, 0.00000000e+00],
       [0.00000000e+00, 7.21500722e-03, 9.90284993e-01, 2.50000000e-03],
       [4.51914099e-03, 2.10613276e-02, 9.63910873e-01, 1.05086580e-02],
       [1.09898225e-01, 5.88882025e-01, 1.82859166e-01, 1.18360584e-01],
       [7.00577201e-03, 8.59858105e-02, 8.85783069e-01, 2.12253487e-02],
       [6.22222222e-03, 7.24514331e-02, 8.96406870e-01, 2.49194749e-02],
       [2.73809524e-03, 3.17460317e-03, 9.88597884e

In [127]:
print('Logarithmic Loss:', metrics.log_loss(y_test, rf_tree_opt.predict_proba(X_test)))

Logarithmic Loss: 0.23443549564435415


### Random Search

In [128]:
rf_random=RandomizedSearchCV(rf_tree,param_distributions=rf_params,n_iter=5,scoring='f1_micro',n_jobs=-1,cv=kf,verbose=3)

In [129]:
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    7.4s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction...
                                                    oob_score=False,
     

In [130]:
rf_random.best_estimator_, rf_random.best_params_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=30, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=5,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False),
 {'n_estimators': 150,
  'min_samples_split': 5,
  'min_samples_leaf': 4,
  'max_depth': 30})

In [131]:
y_pred_cat = rf_random.predict_proba(X_test)
log_loss(y_test, y_pred_cat)

0.2924258638663749

# XGBoost

In [40]:
import os
import gc
import pickle
import time
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [41]:
xgbmodel = XGBClassifier(
    max_depth=10,
    n_estimators=100,
    min_child_weight=0.5, 
    colsample_bytree=0.4, 
    subsample=0.4, 
    eta=0.1,
    seed=30)

xgbmodel.fit(
    X_train, 
    y_train, 
    eval_metric='mlogloss',
    verbose=True)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eta=0.1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=0.5, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=30,
              silent=None, subsample=0.4, verbosity=1)

In [42]:
print('Logarithmic Loss:', metrics.log_loss(y_test, xgbmodel.predict_proba(X_test)))

Logarithmic Loss: 0.03338815379177431


## Optimization

### Grid Search

In [52]:
xgb_params = {'min_child_weight': [0.1, 0.5, 0.7, 0.9, 1], 
              'max_depth': [1, 10, 20, 30, 40],
              'n_estimator': [100, 150, 170, 200, 300],
              'seed': [10, 20, 30, 50, 70]}

kf = KFold(n_splits=5)

In [53]:
xgbmodel_test = XGBClassifier()

In [98]:
grid_searcher_xgb = GridSearchCV(estimator=xgbmodel_test, param_grid=xgb_params, 
                             cv=kf, verbose=1, n_jobs=4)

In [99]:
grid_searcher_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 625 candidates, totalling 3125 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   15.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   32.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 3125 out of 3125 | elapsed:  4.7min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='b...,
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'max_depth': [1, 10, 20, 30, 40],
                         'min_child_weight': [0.1, 0.5, 0.7,

In [100]:
grid_searcher_xgb.best_params_, grid_searcher.best_score_

({'max_depth': 1, 'min_child_weight': 0.1, 'n_estimator': 100, 'seed': 10},
 1.0)

In [101]:
xgbmodel_opt = XGBClassifier(
    max_depth=1,
    n_estimators=100,
    min_child_weight=0.1, 
    colsample_bytree=0.4, 
    subsample=0.4, 
    eta=0.1,
    seed=10)

In [102]:
xgbmodel_opt.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eta=0.1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=0.1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=10,
              silent=None, subsample=0.4, verbosity=1)

In [103]:
log_loss(y_test, xgbmodel_opt.predict_proba(X_test))

0.06993989018811743

### Random search

In [58]:
xgb_random=RandomizedSearchCV(xgbmodel_test,param_distributions=xgb_params,n_iter=5,scoring='f1_micro',n_jobs=-1,cv=kf,verbose=3)

In [59]:
xgb_random.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    4.0s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           object...
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
                   iid='warn', n

In [60]:
xgb_random.best_estimator_, xgb_random.best_params_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=1,
               min_child_weight=0.1, missing=None, n_estimator=170,
               n_estimators=100, n_jobs=1, nthread=None,
               objective='multi:softprob', random_state=0, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=1, seed=10, silent=None,
               subsample=1, verbosity=1),
 {'seed': 10, 'n_estimator': 170, 'min_child_weight': 0.1, 'max_depth': 1})

In [61]:
y_pred_cat = xgb_random.predict_proba(X_test)
log_loss(y_test, y_pred_cat)

0.005057096190709811

# LGBM Model

In [62]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, multilabel_confusion_matrix, f1_score, average_precision_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, KFold
from catboost import CatBoostRegressor, CatBoostClassifier

/Users/amihailovs/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [63]:
lgb_clf = LGBMClassifier(random_state=17, max_depth=-1, 
                          num_leaves=63, n_estimators=300,
                          n_jobs=1)

In [64]:
lgb_clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=300, n_jobs=1, num_leaves=63, objective=None,
               random_state=17, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [65]:
log_loss(y_test, lgb_clf.predict_proba(X_test))

3.963372245652866e-06

## Optimization

### Grid Search

In [66]:
params = {'num_leaves': [25, 30, 35, 70, 100], 
              'max_depth': [-1, 10, 20, 30, 40],
              'n_estimator': [100, 150, 170, 200, 300]}

kf = KFold(n_splits=5)

In [67]:
grid_searcher = GridSearchCV(estimator=lgb_clf, param_grid=params, 
                             cv=kf, verbose=1, n_jobs=4)

In [68]:
grid_searcher.fit(X_train, y_train)

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   16.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   37.1s
[Parallel(n_jobs=4)]: Done 625 out of 625 | elapsed:   58.2s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=300,
                                      n_jobs=1, num_leaves=63, objective=None,
                                      random_state=17, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=4,
             param_grid={'m

In [69]:
grid_searcher.best_params_, grid_searcher.best_score_

({'max_depth': -1, 'n_estimator': 100, 'num_leaves': 25}, 0.9834862385321101)

In [70]:
lgb_clf_opt = LGBMClassifier(random_state=17, max_depth=-1, 
                          num_leaves=25, n_estimators=100,
                          n_jobs=1)

In [71]:
lgb_clf_opt.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=1, num_leaves=25, objective=None,
               random_state=17, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [72]:
log_loss(y_test, lgb_clf_opt.predict_proba(X_test))

1.989696040048749e-05

### Random Search

In [73]:
lgbm = LGBMClassifier()

In [74]:
lgbm_random=RandomizedSearchCV(lgbm,param_distributions=params,n_iter=5,scoring='f1_micro',n_jobs=-1,cv=kf,verbose=3)

In [75]:
lgbm_random.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  25 | elapsed:    1.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.2s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            num_leaves=31, ob...
                                            random_state=None, reg_alpha=0.0,
                                            reg_lambda=0.0, silent=True,
                                            s

In [76]:
lgbm_random.best_estimator_, lgbm_random.best_params_

(LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=20,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimator=150, n_estimators=100, n_jobs=-1, num_leaves=100,
                objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
                silent=True, subsample=1.0, subsample_for_bin=200000,
                subsample_freq=0),
 {'num_leaves': 100, 'n_estimator': 150, 'max_depth': 20})

In [77]:
y_pred_cat = lgbm_random.predict_proba(X_test)
log_loss(y_test, y_pred_cat)

1.989696040048749e-05

# CatBoost

In [78]:
cb_model = CatBoostClassifier(iterations=100,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MultiClass',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [79]:
cb_model.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             use_best_model=True,
             verbose=50)

0:	learn: 1.2773078	test: 1.2831700	best: 1.2831700 (0)	total: 94.2ms	remaining: 9.32s
50:	learn: 0.1480326	test: 0.1756181	best: 0.1756181 (50)	total: 4.43s	remaining: 4.26s
99:	learn: 0.0428343	test: 0.0580716	best: 0.0580716 (99)	total: 6.56s	remaining: 0us

bestTest = 0.05807159405
bestIteration = 99



In [80]:
pred_test_cat = cb_model.predict_proba(X_test)

In [81]:
log_loss(y_test, pred_test_cat)

0.05807159404553854

## Grid Search

In [82]:
params={
    'random_seed' : [10, 15, 20, 30, 40, 50, 60],
    'bagging_temperature' : [0.1, 0.2, 0.3, 0.5, 0.7, 0.9],
    'metric_period' : [20, 30, 40, 50, 70, 90],
    'od_wait' : [5, 10, 20, 25, 30, 40]
}

kf = KFold(n_splits=5)

In [83]:
catboost = CatBoostClassifier()

In [84]:
grid_searcher = GridSearchCV(estimator=catboost, param_grid=params, 
                             cv=kf, verbose=1, n_jobs=4)

In [86]:
grid_searcher.fit(X_train, y_train)

Fitting 5 folds for each of 1512 candidates, totalling 7560 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 17.0min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 30.4min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed: 47.4min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed: 68.3min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed: 101.0min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 150.3min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed: 185.2min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed: 220.9min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed: 262.1min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 306.1min
[Parallel(n_jobs=4)]: Done 7560 out of 7560 | elapsed: 318.7min finished


Learning rate set to 0.076715
0:	learn: 1.2302152	total: 6.77ms	remaining: 6.76s
20:	learn: 0.2453274	total: 77.2ms	remaining: 3.6s
40:	learn: 0.0753633	total: 131ms	remaining: 3.06s
60:	learn: 0.0336588	total: 177ms	remaining: 2.73s
80:	learn: 0.0178124	total: 232ms	remaining: 2.64s
100:	learn: 0.0120536	total: 295ms	remaining: 2.63s
120:	learn: 0.0086573	total: 345ms	remaining: 2.51s
140:	learn: 0.0067326	total: 398ms	remaining: 2.43s
160:	learn: 0.0056469	total: 467ms	remaining: 2.43s
180:	learn: 0.0047766	total: 525ms	remaining: 2.38s
200:	learn: 0.0041502	total: 577ms	remaining: 2.29s
220:	learn: 0.0036648	total: 633ms	remaining: 2.23s
240:	learn: 0.0032710	total: 695ms	remaining: 2.19s
260:	learn: 0.0029527	total: 749ms	remaining: 2.12s
280:	learn: 0.0026902	total: 800ms	remaining: 2.05s
300:	learn: 0.0024717	total: 851ms	remaining: 1.98s
320:	learn: 0.0022858	total: 926ms	remaining: 1.96s
340:	learn: 0.0021258	total: 978ms	remaining: 1.89s
360:	learn: 0.0019871	total: 1.03s	rema

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostClassifier object at 0x7fe1c79cb210>,
             iid='warn', n_jobs=4,
             param_grid={'bagging_temperature': [0.1, 0.2, 0.3, 0.5, 0.7, 0.9],
                         'metric_period': [20, 30, 40, 50, 70, 90],
                         'od_wait': [5, 10, 20, 25, 30, 40],
                         'random_seed': [10, 15, 20, 30, 40, 50, 60]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [87]:
grid_searcher.best_params_, grid_searcher.best_score_

({'bagging_temperature': 0.1,
  'metric_period': 20,
  'od_wait': 5,
  'random_seed': 10},
 1.0)

In [89]:
cb_model_opt = CatBoostClassifier(iterations=100,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MultiClass',
                             random_seed = 10,
                             bagging_temperature = 0.1,
                             od_type='Iter',
                             metric_period = 20,
                             od_wait=5)

In [105]:
cb_model_opt.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             use_best_model=True,
             verbose=20)

0:	learn: 1.3108439	test: 1.3137544	best: 1.3137544 (0)	total: 101ms	remaining: 10s
20:	learn: 0.5221506	test: 0.5703241	best: 0.5703241 (20)	total: 1.63s	remaining: 6.12s
40:	learn: 0.2404346	test: 0.2845909	best: 0.2845909 (40)	total: 2.75s	remaining: 3.96s
60:	learn: 0.1273094	test: 0.1622199	best: 0.1622199 (60)	total: 3.54s	remaining: 2.27s
80:	learn: 0.0786040	test: 0.1074981	best: 0.1074981 (80)	total: 4.52s	remaining: 1.06s
99:	learn: 0.0533090	test: 0.0773945	best: 0.0773945 (99)	total: 5.36s	remaining: 0us

bestTest = 0.07739453791
bestIteration = 99



In [ ]:
#lgb_clf_opt.fit(X_train, y_train)

In [106]:
log_loss(y_test, cb_model_opt.predict_proba(X_test))

0.07739453790663768

## Random Search

In [91]:
random=RandomizedSearchCV(catboost,param_distributions=params,n_iter=5,scoring='f1_micro',n_jobs=-1,cv=kf,verbose=3)

In [92]:
random.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.0min finished


Learning rate set to 0.076715
0:	learn: 1.2146299	total: 5.2ms	remaining: 5.2s
70:	learn: 0.0240744	total: 203ms	remaining: 2.66s
140:	learn: 0.0066997	total: 391ms	remaining: 2.38s
210:	learn: 0.0037046	total: 590ms	remaining: 2.21s
280:	learn: 0.0025635	total: 784ms	remaining: 2.01s
350:	learn: 0.0019532	total: 985ms	remaining: 1.82s
420:	learn: 0.0015854	total: 1.18s	remaining: 1.62s
490:	learn: 0.0013448	total: 1.38s	remaining: 1.43s
560:	learn: 0.0011656	total: 1.58s	remaining: 1.23s
630:	learn: 0.0010322	total: 1.77s	remaining: 1.04s
700:	learn: 0.0009229	total: 1.97s	remaining: 840ms
770:	learn: 0.0008342	total: 2.17s	remaining: 644ms
840:	learn: 0.0007601	total: 2.36s	remaining: 447ms
910:	learn: 0.0006974	total: 2.56s	remaining: 250ms
980:	learn: 0.0006447	total: 2.75s	remaining: 53.3ms
999:	learn: 0.0006318	total: 2.81s	remaining: 0us


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostClassifier object at 0x7fe1c79cb210>,
                   iid='warn', n_iter=5, n_jobs=-1,
                   param_distributions={'bagging_temperature': [0.1, 0.2, 0.3,
                                                                0.5, 0.7, 0.9],
                                        'metric_period': [20, 30, 40, 50, 70,
                                                          90],
                                        'od_wait': [5, 10, 20, 25, 30, 40],
                                        'random_seed': [10, 15, 20, 30, 40, 50,
                                                        60]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='f1_micro', verbose=3)

In [93]:
random.best_estimator_

In [94]:
random.best_params_

{'random_seed': 30,
 'od_wait': 40,
 'metric_period': 70,
 'bagging_temperature': 0.3}

In [95]:
y_pred_cat = random.predict_proba(X_test)
log_loss(y_test, y_pred_cat)

0.001039408317817969

The table provides information on how important are features and how strongly they influence on the model. Based on the scores, the features can be selected for the final version.

# Restructure

Based on the feature importance table, they can be selected for the final model and predictions. 'type_mean_email' exceeds the other importance scores dramaticaly, therefore it overfitting models and the column have to be removed. The features 'Life-Cycle', 'Generation' and 'age' are highly correcleted and represent one thing that is age. High correclated features works worse on tree-based algorithms and increases the chances to be overfitted. Since 'Life-Cycle' has higher score of importance on the model, it is selected for the final model. 'outgoing_min' and 'incoming_min' has no impact on the model and can be removed as well. 'loan_payment_mean' has to be removed as well since potentilly it is unknown how much customer can pay. 

Columns relate to payment amount are fit to the US reality. As results will be predicted for Latvian citizens, features of amount have to be adopted to the country reality. 

In [175]:
#Adoption to LV reality and rounding to 2 numbers after comma
df['amount_mean']=round(df['amount_mean']/10,2)
df['incoming_mean']=round(df['incoming_mean']/10, 2)
df['outgoing_mean']=round(df['outgoing_mean']/10, 2)
df['incoming_max']=round(df['incoming_max']/10, 2)
df['outgoing_max']=round(df['outgoing_max']/10, 2)

In [176]:
from sklearn.model_selection import train_test_split
X = df[['status', 'location', 'amount_mean', 'operation_mean', 'incoming_mean', 
        'outgoing_mean', 'incoming_max', 'outgoing_max', 'Life-Cycle', 
        'Female', 'Male', 'email_types', 'Student', 'Employed', 'Resident']]
Y = df.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [177]:
catboost_final = CatBoostClassifier(iterations=100,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MultiClass',
                             random_seed = 10,
                             bagging_temperature = 0.1,
                             od_type='Iter',
                             metric_period = 20,
                             od_wait=5)

In [178]:
catboost_final.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             use_best_model=True,
             verbose=20)

0:	learn: 1.3127346	test: 1.3151606	best: 1.3151606 (0)	total: 1.13ms	remaining: 112ms
20:	learn: 0.8421578	test: 0.9156679	best: 0.9156679 (20)	total: 876ms	remaining: 3.29s
40:	learn: 0.6910698	test: 0.8124614	best: 0.8124614 (40)	total: 1.82s	remaining: 2.62s
60:	learn: 0.6063280	test: 0.7869256	best: 0.7869256 (60)	total: 2.75s	remaining: 1.76s
80:	learn: 0.5465082	test: 0.7776609	best: 0.7776609 (80)	total: 3.77s	remaining: 883ms
Stopped by overfitting detector  (5 iterations wait)

bestTest = 0.7767423424
bestIteration = 81

Shrink model to first 82 iterations.


In [179]:
pred_test_cat = catboost_final.predict_proba(X_test)

In [180]:
log_loss(y_test, pred_test_cat)

0.7767423424372074

In [184]:
pd.DataFrame(catboost_final.feature_importances_,
             index=X_train.columns, columns=['Feature Importance']).sort_values(
    by='Feature Importance', ascending=False)[:16]

,Feature Importance
status,16.887239
email_types,11.651110
Life-Cycle,10.965092
operation_mean,7.867026
location,7.591448
outgoing_max,7.567692
Female,5.795170
incoming_max,5.474257
amount_mean,5.245389
outgoing_mean,4.920858


# Predictions

In [159]:
real_data = {'CUS_ID': ['001', '002', '003', '004', '005', '006', '007',
                  '008', '009', '010'],
       'Name': ['Arturs', 'Olegs', 'Valters', 'Aleksandrs', 'Madara',
                 'Marija', 'Anna', 'Monta', 'Andris', 'Mihails'],
       'Address': ['Krišjāņa Valdemāra iela 77', 'Ernesta Birznieka-Upīša iela 5-1',
                  'Anniņmuižas bulvāris 38', 'Dammes 4', 'Daugavgrīvas 132',
                  'Prūšu 17a', 'Ezermalas 4a', 'Hipokrāta 23', 'Ilūkstes 34',
                  'Biķernieku 53'],
        'status': ['0', '2', '3', '2', '0', '2', '3',
                  '1', '2', '2'],
        'location': ['1', '1', '1', '1', '1', '1', '1',
                  '1', '1', '1'],
        'amount_mean': ['720', '350', '439', '1198', '678', '358', '2119',
                  '1287', '96', '105'],
        'operation_mean': ['0.23', '1.68', '1.06', '2.12', '3.70', '2.61', '1.98',
                  '0.99', '2.42', '2.32'],
        'incoming_mean': ['600', '150', '430', '1100', '600', '250', '2100',
                  '1200', '79', '98'],
        'outgoing_mean': ['1152', '79', '124', '380', '320', '98', '1020',
                  '987', '45', '38'],
        'incoming_max': ['950', '300', '700', '1450', '1000', '500', '3175',
                  '2150', '200', '150'],
        'outgoing_max': ['200', '150', '300', '900', '765', '179', '2500',
                  '1988', '100', '75'],
        'Life-Cycle': ['1', '1', '0', '2', '3', '1', '2',
                  '2', '1', '3'],
        'Female': ['0', '0', '0', '0', '1', '1', '1',
                  '1', '0', '0'],
        'Male': ['1', '1', '1', '1', '0', '0', '0',
                  '0', '1', '1'],
       'email_types': ['0', '0', '0', '0', '0',
                 '0', '0', '0', '0', '0'],
       'Student':['1', '1', '0', '0', '0', '1', '0', '0', '1', '0'],# Is student?
       'Employed':['1', '0', '0', '1', '0', '0', '1', '1', '0', '0'],# Is employed?
       'Resident':['1', '1', '1', '0', '1', '0', '1', '1', '0', '1']# Is resident
       }

In [160]:
Real_Customers = pd.DataFrame(real_data, columns = ['CUS_ID', 'Name', 'Address', 'status', 'location', 'amount_mean', 
                                               'operation_mean', 'incoming_mean', 'outgoing_mean', 'incoming_max', 
                                               'outgoing_max', 'Life-Cycle', 'Female', 'Male', 'email_types',
                                               'Student', 'Employed', 'Resident'])

In [161]:
Real_Customers

,CUS_ID,Name,Address,status,location,amount_mean,operation_mean,incoming_mean,outgoing_mean,incoming_max,outgoing_max,Life-Cycle,Female,Male,email_types,Student,Employed,Resident
0,001,Arturs,Krišjāņa Valdemāra iela 77,0,1,720,0.23,600,1152,950,200,1,0,1,0,1,1,1
1,002,Olegs,Ernesta Birznieka-Upīša iela 5-1,2,1,350,1.68,150,79,300,150,1,0,1,0,1,0,1
2,003,Valters,Anniņmuižas bulvāris 38,3,1,439,1.06,430,124,700,300,0,0,1,0,0,0,1
3,004,Aleksandrs,Dammes 4,2,1,1198,2.12,1100,380,1450,900,2,0,1,0,0,1,0
4,005,Madara,Daugavgrīvas 132,0,1,678,3.70,600,320,1000,765,3,1,0,0,0,0,1
5,006,Marija,Prūšu 17a,2,1,358,2.61,250,98,500,179,1,1,0,0,1,0,0
6,007,Anna,Ezermalas 4a,3,1,2119,1.98,2100,1020,3175,2500,2,1,0,0,0,1,1
7,008,Monta,Hipokrāta 23,1,1,1287,0.99,1200,987,2150,1988,2,1,0,0,0,1,1
8,009,Andris,Ilūkstes 34,2,1,96,2.42,79,45,200,100,1,0,1,0,1,0,0
9,010,Mihails,Biķernieku 53,2,1,105,2.32,98,38,150,75,3,0,1,0,0,0,1


In [162]:
Real_Customers.to_excel (r'/Users/amihailovs/Desktop/Business Analytics/real_customers.xlsx', index = False, header=True)

In [163]:
Prediction_cols = ['status', 'location', 'amount_mean', 'operation_mean', 
                   'incoming_mean', 'outgoing_mean', 'incoming_max','outgoing_max', 
                   'Life-Cycle', 'Female', 'Male', 'email_types', 'Student', 'Employed', 'Resident']

In [164]:
real_pred_cat = random.predict_proba(Real_Customers[Prediction_cols])

In [165]:
prediction_columns = ['car', 'debt_consolidation', 'home', 'home_improvement']

In [166]:
Predictions = round(pd.DataFrame(real_pred_cat, columns=prediction_columns), 2)

In [167]:
Predictions

,car,debt_consolidation,home,home_improvement
0,0.02,0.14,0.78,0.07
1,0.07,0.14,0.64,0.15
2,0.01,0.01,0.93,0.05
3,0.00,0.04,0.95,0.01
4,0.01,0.09,0.88,0.01
5,0.13,0.31,0.25,0.31
6,0.00,0.05,0.92,0.02
7,0.01,0.09,0.78,0.12
8,0.10,0.13,0.63,0.14
9,0.04,0.14,0.80,0.02


In [168]:
Real_Customers

,CUS_ID,Name,Address,status,location,amount_mean,operation_mean,incoming_mean,outgoing_mean,incoming_max,outgoing_max,Life-Cycle,Female,Male,email_types,Student,Employed,Resident
0,001,Arturs,Krišjāņa Valdemāra iela 77,0,1,720,0.23,600,1152,950,200,1,0,1,0,1,1,1
1,002,Olegs,Ernesta Birznieka-Upīša iela 5-1,2,1,350,1.68,150,79,300,150,1,0,1,0,1,0,1
2,003,Valters,Anniņmuižas bulvāris 38,3,1,439,1.06,430,124,700,300,0,0,1,0,0,0,1
3,004,Aleksandrs,Dammes 4,2,1,1198,2.12,1100,380,1450,900,2,0,1,0,0,1,0
4,005,Madara,Daugavgrīvas 132,0,1,678,3.70,600,320,1000,765,3,1,0,0,0,0,1
5,006,Marija,Prūšu 17a,2,1,358,2.61,250,98,500,179,1,1,0,0,1,0,0
6,007,Anna,Ezermalas 4a,3,1,2119,1.98,2100,1020,3175,2500,2,1,0,0,0,1,1
7,008,Monta,Hipokrāta 23,1,1,1287,0.99,1200,987,2150,1988,2,1,0,0,0,1,1
8,009,Andris,Ilūkstes 34,2,1,96,2.42,79,45,200,100,1,0,1,0,1,0,0
9,010,Mihails,Biķernieku 53,2,1,105,2.32,98,38,150,75,3,0,1,0,0,0,1


In [169]:
full_cust_list = pd.concat([Real_Customers, Predictions], axis=1, join='inner')

In [170]:
full_cust_list[['Name', 'car', 'debt_consolidation', 'home', 'home_improvement']]

,Name,car,debt_consolidation,home,home_improvement
0,Arturs,0.02,0.14,0.78,0.07
1,Olegs,0.07,0.14,0.64,0.15
2,Valters,0.01,0.01,0.93,0.05
3,Aleksandrs,0.00,0.04,0.95,0.01
4,Madara,0.01,0.09,0.88,0.01
5,Marija,0.13,0.31,0.25,0.31
6,Anna,0.00,0.05,0.92,0.02
7,Monta,0.01,0.09,0.78,0.12
8,Andris,0.10,0.13,0.63,0.14
9,Mihails,0.04,0.14,0.80,0.02
